# Posición de la ISS

En este notebook determinamos la posición de la ISS a partir de los elementos orbitales para cualquier instante y lo representamos sobre un mapa

## Descarga de los Elementos orbitales 

Para la descarga debemos tener una cuenta en https://www.space-track.org Los elementos orbitales osculadores se distribuyen en formato TLE http://en.wikipedia.org/wiki/Two-line_element_set y contienen los datos necesarios para el cálculo de la posición de un satélite en un momento determinado. Estos elementos son válidos para una fecha próxima a la de osculación.

In [ ]:
import urllib, urllib2, cookielib
import datetime

noradId = '25544'       # NORADID de la ISS
fecha = '2016/06/15'    # Fecha de osculación

baseURL = 'https://www.space-track.org'
username = 'jgcasta'
password = '123456789012345A'

d = datetime.datetime.strptime(fecha, "%Y/%m/%d")
d1 = d + datetime.timedelta(days=1)
dstr = d.strftime("%Y-%m-%d")
d1str = d1.strftime("%Y-%m-%d")

print "Connecting..."
cj = cookielib.CookieJar()
opener = urllib2.build_opener(urllib2.HTTPCookieProcessor(cj))
parameters = urllib.urlencode({'identity': username ,'password': password})
opener.open(baseURL + '/ajaxauth/login', parameters)
queryString = baseURL +"/basicspacedata/query/class/tle/format/tle/NORAD_CAT_ID/"
queryString = queryString + noradId+"/EPOCH/"+dstr+"%2000:00:00--"+d1str+"%2000:00:00"
resp = opener.open(queryString)

TLEs = resp.read()
print "---------------------------------TLE---------------------------------\n"
while tle in TLEs:
    print tle

opener.close()



## Cálculo de las efemérides

Las efemérides ofrecen datos válidos para un observador sobre la superficie terrestre. Estas se denominan topocéntricas, y para su cálculo necesitamos definir nuestra posición y el instante. ¿Gráfico?

In [9]:
import math
import time
import ephem

obs = ephem.Observer()
obs.lon = '-3.5'
obs.lat = '40.5'
obs.elevation = 650

sat = ephem.readtle('ISS',
    '1 25544U 98067A   16167.93013459  .00006642  00000-0  10575-3 0  9994',
    '2 25544  51.6430  64.3012 0000343   7.3896  56.7230 15.54575408  4797'
)

obs.date = datetime.utcnow()
sat.compute(home)

print sat

<ephem.EarthSatellite 'ISS' at 0x7efbf00d5468>


Además de las topocéntricas lo que nos interesa es conocer la posición que ocupará el satélite a lo largo de su órbita, para planificar la toma de imágenes. Para ello calculamos el punto subterrestre. Este viene definido por las coordenadas sobre la superficie, desde las que satélite aparece en el nadir. Las calculamos para un día completo y las proyectamos sobre el mapa. 

Para planificar cuándo es necesario realizar las fotografías, se compara la posición subterrestre del satélite con los diferentes puntos que se quieren fotografiar. Los momentos de paso se comunican al centro de la misión para que puedan ser planinficados dentro de las actividades diarias